In [3]:
import pandas as pd

# df = pd.read_csv('/Users/erho/analyses/escalation/BERT/officer_utterance_45words.csv', delimiter=',', low_memory=False)
# df = pd.read_csv('/Users/erho/analyses/escalation/BERT/officer_utterance_45words_GF.csv', delimiter=',', low_memory=False)
# df = pd.read_csv('/Users/erho/analyses/escalation/BERT/eliminate_cutoffs_sentences.csv', delimiter=',', low_memory=False) # 180_45_sentences.csv
# df = pd.read_csv('/users/erho/analyses/escalation/BERT/5050testtrain/45_stops.csv', delimiter=',', low_memory=False)
# df = pd.read_csv('/Users/erho/analyses/escalation/BERT_old_FIXED/45words_stops.csv', delimiter=',', low_memory=False)

df = pd.read_csv('/content/drive/MyDrive/meme/meme700march.csv', delimiter=',', low_memory=False)

In [4]:
df_original = df
none = df.loc[df['escalation'] == 0].sample(200)
esc = df.loc[df['escalation'] == 1].sample(200)

df = pd.concat([esc,none])
df = df.sample(frac=1) # shuffle rows
df.to_csv('test_sample_new_t1a_stops.csv')

# select all utterances not part of N=250 (120, 130)
df_training = df_original[~df_original.isin(df)].dropna(how = 'all')
print(len(df_training))
df_training.to_csv('pretraining_sample_new_t1a_stops.csv')

984


In [1]:
from google.colab import drive  ## Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/meme')



In [ ]:
ls

BERT_EUG.ipynb                    pretraining_sample_new_t1a_stops.csv
meme_ig1.csv                      __pycache__/
out_importance_new_t1a_stops.tsv  runs/
out_metrics_new_t1a_stops.tsv     test_sample_new_t1a_stops.csv
out_viz_t1a_stops.html            util_old_FIXED.py


In [5]:
pretraining_texts = df_training['truncated_utterance'].tolist()
# list of strings, X values for classification problem
classification_texts = df['truncated_utterance'].tolist()
# Y for classification problem
classification_Y = df['escalation'].tolist()

WORKING_DIR = '/content/drive/MyDrive/meme/'
# token importance will be written here
OUT_IMPORTANCE = WORKING_DIR + 'out_importance_new_t1a_stops.tsv'
# metrics will be written here (and printed)
OUT_METRICS = WORKING_DIR + 'out_metrics_new_t1a_stops.tsv'
# Where to write vizualizations
OUT_VIZ = WORKING_DIR + 'out_viz_t1a_stops.html'


LANGUAGE_MODELING_EPOCHS = 2 # turn this to zero for no pretraining
CLASSIFICATION_EPOCHS = 4
XVAL_SPLITS = 3

In [6]:
pip install transformers

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 29.7 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [9]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, DistilBertForMaskedLM
from torch.utils.data import Dataset, TensorDataset
import torch
from transformers import Trainer, DataCollatorForLanguageModeling, TrainingArguments
from sklearn.model_selection import KFold
from collections import defaultdict
import numpy as np


import util_old_FIXED as util


In [10]:
if LANGUAGE_MODELING_EPOCHS > 0:
    dataset, tokenizer = util.build_dataset(pretraining_texts)
    mlm_model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")
    result, mlm_model = util.finetune_model(
        model=mlm_model, 
        train_dataset=dataset, 
        tokenizer=tokenizer, 
        working_dir=WORKING_DIR,
        mlm=True,
        epochs=LANGUAGE_MODELING_EPOCHS)


############################################################
# Classification


cls_model = DistilBertForSequenceClassification.from_pretrained(
            "distilbert-base-uncased",
            num_labels=2,
            output_attentions=False,
            output_hidden_states=False)
if LANGUAGE_MODELING_EPOCHS > 0:
    cls_model = util.transfer_parameters(mlm_model, cls_model)


def select_indices(idx, l):
    out = []
    idx = set(idx)
    for i, j in enumerate(l):
        if i in idx:
            out.append(j)
    return out


eval_results = None
attr_results = None
vizs = []
kf = KFold(n_splits=XVAL_SPLITS)
print('## CLASSIFICATION TRAINING')
for i, (train_idx, test_idx) in enumerate(kf.split(classification_texts)):
    print('\tKF SPLIT %d' % i)
    train_dataset, _ = util.build_dataset(
        texts=select_indices(train_idx, classification_texts),
        labels=select_indices(train_idx, classification_Y))
    test_dataset, tokenizer = util.build_dataset(
        texts=select_indices(test_idx, classification_texts),
        labels=select_indices(test_idx, classification_Y))

    train_result, cls_model = util.finetune_model(
        model=cls_model, 
        train_dataset=train_dataset, 
        # eval_dataset=test_dataset,
        tokenizer=tokenizer, 
        working_dir=WORKING_DIR,
        mlm=False,
        epochs=CLASSIFICATION_EPOCHS)

    report, viz, t2a = util.eval(
        cls_model, test_dataset, tokenizer)
    print('REPORT:  ', report)
    vizs += viz

    if eval_results is None:
        eval_results = report
    else:
        for k, v in report.items():
            eval_results[k] += v

    if attr_results is None:
        attr_results = t2a
    else:
        for k, v in t2a.items():
            attr_results[k] += v

for k, v in eval_results.items():
    eval_results[k] /= float(XVAL_SPLITS)

############################################################
# Writing to output

print('## DONE! writing to output.')
print('## metrics:')
print(eval_results)

with open(OUT_VIZ, 'w') as f:
    f.write('\n'.join(vizs))

attr_rows = []
for k in attr_results:
    attr_rows.append({
        'token': k,
        '1': np.mean(attr_results[k]),
        'N': len(attr_results[k])
    })

df_attr = pd.DataFrame(attr_rows)
df_attr = df_attr.sort_values(by=['1', 'N'], ascending=False)
df_attr.to_csv(OUT_IMPORTANCE, sep='\t')

s = ''
for k, v in eval_results.items():
    if k == 'confusion-matrix':
        v = v.tolist()
    s += '%s\t%s\n' % (k, str(v))

with open(OUT_METRICS, 'w') as f:
    f.write(s)


Output hidden; open in https://colab.research.google.com to view.

In [8]:
!pip install captum

     |████████████████████████████████| 1.4 MB 3.8 MB/s 


In [ ]:
""